In [1]:
from rdflib import URIRef, BNode, Literal
from rdflib.namespace import RDF, RDFS
from pyfuseki import FusekiUpdate, FusekiQuery
from SPARQLWrapper import SPARQLWrapper, JSON
from api.src.schemas.bibframe.work import Work_Schema

In [2]:
acervoUpdate = FusekiUpdate('http://localhost:3030', 'acervo')
acervoQuery = FusekiQuery('http://localhost:3030', 'acervo')

Title

In [3]:
obj = {
    "title": "TESTE",
    "typeLabel": "Texto",
    "contribution": "Popper, Karl R. (Karl Raimund), 1902-1994",
    "serie": "Pensamento científico",
    "hasInstance": "https://bibliokeia.com/resources/instance/bk-3",
    "instanceID": "bk-3",
    "contentType": "Texto",
    "mainTitle": "TESTE",
    "subtitle": "",
    "contributionAgent": "Popper, Karl R. (Karl Raimund), 1902-1994",
    "contributionRole": "Autor",
    "contributionRoleUri": "http://id.loc.gov/vocabulary/relators/aut",
    "contributionUri": "https://bibliokeia.com/authorities/names/n80032184",
    "contributionID": "n80032184",
    "subjects": [
        {
            "uri": "https://bibliokeia.com/authorities/subjects/sh85084414",
            "label": "Metodologia"
        },
        {
            "uri": "https://bibliokeia.com/authorities/subjects/sh85118553",
            "label": "Ciência"
        }
    ],
    "language": "",
    "languageCode": "",
    "cdd": "548.2",
    "cutter": "A587s",
    "serieURI": "https://bibliokeia.com/resources/hub/bk-1"
}

work = Work_Schema(**obj)
work


Work_Schema(work_id=None, contentType='Texto', mainTitle='TESTE', subtitle='', variantTitle=None, contributionAgent='Popper, Karl R. (Karl Raimund), 1902-1994', contributionID='n80032184', contributionRole='Autor', contributionRoleUri='http://id.loc.gov/vocabulary/relators/aut', contributionUri='https://bibliokeia.com/authorities/names/n80032184', subjects=[Subject(label='Metodologia', uri='https://bibliokeia.com/authorities/subjects/sh85084414'), Subject(label='Ciência', uri='https://bibliokeia.com/authorities/subjects/sh85118553')], language='', languageCode='', cdd='548.2', cutter='A587s', serie='Pensamento científico', serieURI='https://bibliokeia.com/resources/hub/bk-1')

In [27]:
acervoUpdate = FusekiUpdate('http://localhost:3030', 'acervo')
acervoQuery = FusekiQuery('http://localhost:3030', 'acervo')

In [63]:
r = acervoQuery.run_sparql("""PREFIX work: <https://bibliokeia.com/resources/work/>
PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
ASK { 
  graph work:bk-2
  {work:bk-2 rdf:type bf:Texto}}""")
r.convert()

{'head': {}, 'boolean': True}

In [80]:
acervoUpdate = FusekiUpdate('http://localhost:3030', 'acervo')
acervoQuery = FusekiQuery('http://localhost:3030', 'acervo')

def EditTypeWork(contentType, bkID):
        
        prefix = """PREFIX work: <https://bibliokeia.com/resources/work/>
            PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>
            PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>"""
        where = "{work:"+bkID+" rdf:type bf:"+contentType+"}"
            
        ask = prefix+"ASK { graph work:bk-2 "+where+" }"

        response = acervoQuery.run_sparql(ask)
        response = response.convert()
    
        if not response['boolean']:
            up = prefix+"WITH work:"+bkID+"""
            DELETE { work:"""+bkID+""" rdf:type bf:Texto }
            INSERT { work:"""+bkID+""" rdf:type bf:Texto }
            WHERE """+where
            acervoUpdate.run_sparql(up)
            print("PRECISA ALTERAR")
        else:
            print("NÃO PRECISA ALTERAR NADA")
    
EditTypeWork("Textox", "bk-2")


PRECISA ALTERAR


In [99]:
def Edit(contentType, bkID):

    prefix = """PREFIX work: <https://bibliokeia.com/resources/work/>
                PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>
                PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>"""
    where = "{work:"+bkID+" rdf:type ?o}"
                
    select = prefix+"SELECT * { graph work:bk-2 {work:"+bkID+" rdf:type ?o} }"

    response = acervoQuery.run_sparql(select)
    response = response.convert()
    bindings = response['results']['bindings']

    for binding in bindings:
        uri = binding['o']['value']
        content = uri.split("/")[-1]
        if content != 'Work' and content != contentType:
            where = "{ work:"+bkID+" rdf:type bf:"+content+" }"
            up = prefix+"WITH work:"+bkID+"""
            DELETE """+where+"""
            INSERT { work:"""+bkID+" rdf:type bf:"+contentType+""" }
            WHERE """+where
            acervoUpdate.run_sparql(up)


Edit("Text", "bk-2")

TITLE

In [7]:
def EditTitle(mainTitle, bkID):

    acervoUpdate = FusekiUpdate('http://localhost:3030', 'acervo')
    acervoQuery = FusekiQuery('http://localhost:3030', 'acervo')

    prefix = """PREFIX work: <https://bibliokeia.com/resources/work/>
                PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>
                PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>"""
    # Main Title
    ask = prefix+""" ASK { graph work:"""+bkID+"""
            {work:"""+bkID+""" bf:title ?o .
            ?o bf:mainTitle '"""+mainTitle+"' }}"""
    response = acervoQuery.run_sparql(ask)
    response = response.convert()

    if not response['boolean']:  
        up = prefix+"WITH work:"+bkID+"""
                    DELETE {work:"""+bkID+""" bf:title ?o .
                        ?o bf:mainTitle ?t }
                    INSERT {work:"""+bkID+""" bf:title ?o .
                        ?o bf:mainTitle '"""+mainTitle+"""' }
                    WHERE {work:"""+bkID+""" bf:title ?o .
                        ?o bf:mainTitle ?t }"""

        acervoUpdate.run_sparql(up)

        label = prefix+"WITH work:"+bkID+"""
                    DELETE {work:"""+bkID+""" bf:title ?o .
                        ?o rdfs:label ?t }
                    INSERT {work:"""+bkID+""" bf:title ?o .
                        ?o rdfs:label '"""+mainTitle+"""' }
                    WHERE {work:"""+bkID+""" bf:title ?o .
                        ?o rdfs:label ?t }"""
        acervoUpdate.run_sparql(label)
    
    # Subtitle
    askSubtitle = prefix+""" ASK { graph work:"""+bkID+"""
            {work:"""+bkID+""" bf:title ?o .
            ?o bf:subtitle '"""+mainTitle+"' }}"""
    
    askResponse = acervoQuery.run_sparql(askSubtitle)
    askResponse = askResponse.convert()
    if not askResponse['boolean']: 
        print('PRECISA INCLUIR SUBTITULO')
    else:
        print('NÃO PRECISA FAZER NADA')



EditTitle("TITULO_5", "bk-2")

PRECISA INCLUIR SUBTITULO


In [6]:
acervoUpdate = FusekiUpdate('http://localhost:3030', 'acervo')
acervoQuery = FusekiQuery('http://localhost:3030', 'acervo')

In [65]:
def EditSubtitle(subtitle, title,  bkID):    

    acervoUpdate = FusekiUpdate('http://localhost:3030', 'acervo')
    acervoQuery = FusekiQuery('http://localhost:3030', 'acervo')

    prefix = """PREFIX work: <https://bibliokeia.com/resources/work/>
                PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>
                PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
                PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>"""
    
    subtitleSame = prefix+""" ASK { graph work:"""+bkID+"""
            {work:"""+bkID+""" bf:title ?o .
            ?o bf:subtitle '"""+subtitle+"' }}"
    
    askSubtitleSame = acervoQuery.run_sparql(subtitleSame)
    askSubtitleSame = askSubtitleSame.convert()

    if not askSubtitleSame['boolean']:
        subtitleExist = prefix+""" ASK { graph work:"""+bkID+"""
            {work:"""+bkID+""" bf:title ?o .
            ?o bf:subtitle ?subtitle }}"""
        askSubtitleExist = acervoQuery.run_sparql(subtitleExist)
        askSubtitleExist = askSubtitleExist.convert()

        if askSubtitleExist['boolean']:
            up = prefix+"WITH work:"+bkID+"""
                DELETE {work:"""+bkID+""" bf:title ?o .
                        ?o ?p ?t }
                    INSERT {work:"""+bkID+""" bf:title ?o .
                    ?o rdf:type bf:Title .
                        ?o bf:mainTitle '"""+title+"""' .
                        ?o bf:subtitle '"""+subtitle+"""'
                          }
                    WHERE {work:"""+bkID+""" bf:title ?o .
                        ?o ?p ?t }"""
            acervoUpdate.run_sparql(up)
        else:
            up = prefix+"INSERT DATA { GRAPH work:"+bkID+" { work:"+bkID+"""  bf:title [
                        rdf:type bf:Title;
                        bf:mainTitle '"""+title+"""' ;
                        bf:subtitle '"""+subtitle+"""' ] . } }"""
            acervoUpdate.run_sparql(up)


EditSubtitle("Subtitulo", "Titulo", "bk-2")  

SUBTITULO NÃO EXISTE


DELETE

In [64]:
prefix = """PREFIX work: <https://bibliokeia.com/resources/work/>
                PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>
                PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
                PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>"""

up = prefix+"WITH work:"+bkID+"""
                DELETE {work:"""+bkID+""" bf:title ?o .
                        ?o ?p ?t }
                WHERE {work:"""+bkID+""" bf:title ?o .
                        ?o ?p ?t }"""

response = acervoUpdate.run_sparql(up)
response = response.convert()
response

{'statusCode': 200, 'message': 'Update succeeded'}

AUTOR

In [7]:
contributionUri = work.contributionUri

In [6]:
def EditContributo(work, bkID):

    prefix = """PREFIX work: <https://bibliokeia.com/resources/work/>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>"""

    askAgent = prefix+"""
ASK { graph work:bk-2
  {work:bk-2 bf:contribution ?o .
    ?o bf:agent <"""+work.contributionUri+"""> }}"""

    response = acervoQuery.run_sparql(askAgent)
    response = response.convert()
    if not response['boolean']:
        up = prefix+"WITH work:"+bkID+"""
                DELETE {work:"""+bkID+""" bf:contribution ?o .
                        ?o ?p ?agent }
                INSERT {work:"""+bkID+""" bf:contribution ?o .
                ?o rdf:type bf:Contribution .
                ?o rdfs:label '"""+work.contributionAgent+"""' .
                ?o bf:role '"""+work.contributionRoleUri+"""' .
                ?o bf:agent <"""+work.contributionUri+"""> }
                WHERE {work:"""+bkID+""" bf:contribution ?o .
                        ?o ?p ?agent }"""
        acervoUpdate.run_sparql(up)

        

    
work.contributionUri = 'https://bibliokeia.com/authorities/names/n80032184'
EditContributo(work, 'bk-2')


In [4]:
work.contributionRoleUri

'http://id.loc.gov/vocabulary/relators/aut'

In [19]:
'https://bibliokeia.com/authorities/names/n80002329'
https://bibliokeia.com/authorities/names/n80032184

In [ ]:
https://bibliokeia.com/authorities/names/n80032184

In [29]:
work

Work_Schema(work_id=None, contentType='Texto', mainTitle='TESTE', subtitle='', variantTitle=None, contributionAgent='Popper, Karl R. (Karl Raimund), 1902-1994', contributionID='n80032184', contributionRole='Autor', contributionUri='https://bibliokeia.com/authorities/names/n80002329', subjects=[Subject(label='Metodologia', uri='https://bibliokeia.com/authorities/subjects/sh85084414'), Subject(label='Ciência', uri='https://bibliokeia.com/authorities/subjects/sh85118553')], language='', languageCode='', cdd='548.2', cutter='A587s', serie='Pensamento científico', serieURI='https://bibliokeia.com/resources/hub/bk-1')

In [10]:
bkID = 'BK-2'
contributionUri = 'https://bibliokeia.com/authorities/names/n80002329'
up = prefix+"WITH work:"+bkID+"""
                DELETE {work:"""+bkID+""" bf:contribution ?o .
                        ?o bf:agent ?uri }
                INSERT {work:"""+bkID+""" bf:contribution ?o .
                   ?o bf:agent <"""+contributionUri+"""> }
                WHERE {work:"""+bkID+""" bf:contribution ?o .
                        ?o bf:agent ?uri }"""
acervoUpdate.run_sparql(up)

NameError: name 'prefix' is not defined

In [8]:
askAgent = """PREFIX work: <https://bibliokeia.com/resources/work/>
PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>
ASK { graph work:bk-2
  {work:bk-2 bf:contribution ?o .
    ?o bf:agent <"""+contributionUri+"""> }}"""

response = acervoQuery.run_sparql(askAgent)
response = response.convert()
response



{'head': {}, 'boolean': True}